In [ ]:
import numpy as np
import keras
import cv2
from keras import backend as k
from keras.models import Sequential,Model,load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization, UpSampling2D, Dropout, Flatten, Dense, Input, LeakyReLU, Conv2DTranspose,AveragePooling2D, Concatenate
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image
from skimage import color, io
from zipfile import ZipFile 
from io import BytesIO
import matplotlib.pyplot as plt

np.random.seed(1)
set_random_seed(1)

In [ ]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  
  def __init__(self, list_IDs, zip_file, batch_size=32, dim=(256, 256), n_channels=3, shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.list_IDs = list_IDs
    self.zip_file = zip_file
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.on_epoch_end()            # shuffle data on the beginning

  def __len__(self):
    'Denotes the number of batches per epoch'
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    y = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
    X = np.empty((self.batch_size, *self.dim, 1), dtype=np.float32)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
      img_rgb = np.array(Image.open(BytesIO(self.zip_file.read(ID))).convert('RGB')) / 255
      img_lab = color.rgb2lab(img_rgb)
      img_lab[:,:,0] = img_lab[:,:,0] / 50 - 1
      img_lab[:,:,0] = img_lab[:,:,0]

      img_lab[:,:,1:] = (img_lab[:,:,1:] + 128) / 255 * 2 - 1
      
      y[i,] = img_lab
      X[i,] = color.rgb2gray(img_rgb)[...,np.newaxis]*2 - 1
            
    return X, y


In [ ]:
data_path = '/content/drive/My Drive/data_b7b_elcima/colored.zip'
zip_data =  ZipFile(data_path, 'r')
ids = zip_data.namelist()[1:]
np.random.shuffle(ids)
dev_length = int(0.1 * len(ids))
test_length = int(0.05 * len(ids))
test_ids = ids[:test_length]
dev_ids = ids[test_length:dev_length]
train_ids = ids[dev_length:]

In [ ]:
batch_size = 32

test_generator = DataGenerator(test_ids, zip_data, batch_size=batch_size)
training_generator = DataGenerator(train_ids, zip_data, batch_size=batch_size)
validation_generator = DataGenerator(dev_ids, zip_data, batch_size=batch_size)